In [87]:
import os
import subprocess
import tskit
import msprime
import pyslim

In [116]:
SLIM_SCRIPT = """
    initialize() {{
       setSeed({seed});
       initializeSLiMModelType("nonWF");
       initializeSex("A");
       initializeTreeSeq();
       initializeMutationRate(0.0);
       initializeMutationType("m1", 0.5, "f", 0.0);
       initializeGenomicElementType("g1", m1, 1.0);
       initializeGenomicElement(g1, 0, 1e8-1);
       initializeRecombinationRate(1e-8);    
       defineConstant("K", 1000);
    }}

    reproduction(p1, "F") {{
       subpop.addCrossed(individual,
                         subpop.sampleIndividuals(1, sex="M"));
    }}

    1 early() {{
       sim.addSubpop("p1", K);
    }}

    early() {{
       p1.fitnessScaling = K / p1.individualCount;
    }}

    1000 late() {{
       sim.treeSeqOutput("/tmp/{name}.trees");
    }}
"""

In [117]:
def slim_func(name, seed):
    """
    Runs a simple simulation from a starting seed, 
    saves a treesequence, loads the finished ts with
    pyslim, and returns the SlimTreeSequence object.
    """
    # write slim script to temp file
    script_path = os.path.join("/tmp", name + ".slim")
    with open(script_path, 'w') as out:
        out.write(SLIM_SCRIPT.format(seed=seed, name=name))
        
    # run slim
    subprocess.run(
        ['slim', script_path], 
        stdout=subprocess.DEVNULL, 
        stderr=subprocess.STDOUT, 
        check=True,
    )
    
    # load the tree sequence
    mts = pyslim.load(f'/tmp/{name}.trees')
    return mts

In [118]:
# simulate 100 gens of selection on a large genome
ts0 = slim_func('ts0', 123)
ts1 = slim_func('ts1', 321)

In [119]:
ts0

In [120]:
# merge the two ts into one ts with 2 populations
merged_ts = pyslim.SlimTreeSequence(
    ts0.union(
        ts1, 
        node_mapping=[tskit.NULL for i in range(ts1.num_nodes)],
        add_populations=False,
    )
)

In [138]:
for pop in ts0.populations():
    print(pop)

{'id': 0, 'metadata': None}
{'id': 1, 'metadata': {'slim_id': 1, 'selfing_fraction': 0.0, 'female_cloning_fraction': 0.0, 'male_cloning_fraction': 0.0, 'sex_ratio': 0.0, 'bounds_x0': 0.0, 'bounds_x1': 1.0, 'bounds_y0': 0.0, 'bounds_y1': 1.0, 'bounds_z0': 0.0, 'bounds_z1': 1.0, 'migration_records': []}}


In [140]:
proc = subprocess.Popen(['ls'])

In [151]:
import pydantic
import toytree

In [128]:
demography = msprime.Demography()
demography.add_population()

# NEW MSP
pyslim.SlimTreeSequence(
    msprime.sim_ancestry(
        initial_state=merged_ts,
        demography=demography,
    )
)

ValueError: Must specify demography for initial_state with > 1 population

In [126]:
# OLD MSPRIME METHOD
pop_configs = [
    msprime.PopulationConfiguration(initial_size=1000)
    for _ in range(merged_ts.num_populations)
]

rts = merged_ts.recapitate(
    population_configurations=pop_configs,
    migration_matrix=[
        [0.0, 0.0,],
        [0.0, 0.0,],
    ],
    recombination_rate=1e-8,
    random_seed=4,
)

In [122]:
# add mutations to the final treeseq
ts = pyslim.SlimTreeSequence(
    msprime.sim_mutations(rts, discrete_genome=True, rate=1e-8, random_seed=7))

In [123]:
ts = ts.simplify()

In [124]:
ts